In [1]:
import pandas as pd
import numpy as np

# Load your data
df = pd.read_csv('data.csv')

# Display raw column names
print("Raw Column Names:", df.columns.tolist())

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Keep only the relevant columns
relevant_columns = [
    'name', 'reign', 'days', 'date', 'event', 'location', 'link', 'belt',
    'date_of_birth', 'date_of_death', 'cause_of_death'
]
df = df[[col for col in relevant_columns if col in df.columns]]

# Check for required 'belt' column
if 'belt' not in df.columns:
    print("Error: 'belt' column not found. Available columns:", df.columns)
    exit()
else:
    print("'belt' column exists, proceeding with cleaning...")

# Filter to only relevant titles
target_titles = [
    "wwe championship",
    "wwe us championship",
    "wwe women's championship",
    "wwe raw women's championship",
    "wwe smackdown women's championship"
]
df = df[df['belt'].str.lower().isin(target_titles)]

# Convert date columns
df['reign_start'] = pd.to_datetime(df['date'], errors='coerce')
df['reign_end'] = pd.to_datetime(df['reign'], errors='coerce')  # Assuming 'reign' holds end date

# Fill ongoing reigns with today's date
today = pd.to_datetime("today")
df['reign_end'] = df['reign_end'].fillna(today)

# Calculate duration of reigns
df['reign_duration'] = (df['reign_end'] - df['reign_start']).dt.days

# Sort and calculate gap between reigns
df = df.sort_values(by=['name', 'reign_start'])
df['prev_reign_end'] = df.groupby('name')['reign_end'].shift(1)
df['gap_between_reigns'] = (df['reign_start'] - df['prev_reign_end']).dt.days

# Aggregate core summary features
summary = (
    df.groupby(['name', 'belt'])
    .agg(
        total_reigns=('belt', 'count'),
        total_days_as_champion=('reign_duration', 'sum'),
        average_reign_duration=('reign_duration', 'mean'),
        longest_single_reign=('reign_duration', 'max'),
        shortest_single_reign=('reign_duration', 'min'),
        most_recent_reign=('reign_start', 'max'),
        first_reign=('reign_start', 'min'),
        most_recent_reign_end=('reign_end', 'max')
    )
    .reset_index()
)

# Additional metrics
summary['days_since_last_reign'] = (today - summary['most_recent_reign']).dt.days
title_diversity = df.groupby('name')['belt'].nunique().reset_index(name='title_diversity')
avg_gap = df.groupby('name')['gap_between_reigns'].mean().reset_index(name='avg_gap_between_reigns')

career_span = (
    df.groupby('name')
    .agg(first_title_date=('reign_start', 'min'), last_title_date=('reign_end', 'max'))
    .reset_index()
)
career_span['career_span_days'] = (career_span['last_title_date'] - career_span['first_title_date']).dt.days

total_titles = df.groupby('name')['belt'].nunique()
total_reigns = df.groupby('name')['belt'].count()
avg_reigns_per_title = (total_reigns / total_titles).reset_index(name='avg_reigns_per_title')

# Merge everything
summary = summary.merge(title_diversity, on='name', how='left')
summary = summary.merge(avg_gap, on='name', how='left')
summary = summary.merge(career_span[['name', 'career_span_days']], on='name', how='left')
summary = summary.merge(avg_reigns_per_title, on='name', how='left')

# Round numeric values
summary['average_reign_duration'] = summary['average_reign_duration'].round(1)
summary['avg_gap_between_reigns'] = summary['avg_gap_between_reigns'].round(1)
summary['avg_reigns_per_title'] = summary['avg_reigns_per_title'].round(2)

# Save to CSV
summary.to_csv('cleaned_wrestling_data.csv', index=False)

print("Data cleaned and saved to 'cleaned_wrestling_data.csv'")



Raw Column Names: ['name', 'reign', 'days', 'date', 'event', 'location', 'link', 'belt', 'date_of_birth', 'date_of_death', 'cause_of_death']
'belt' column exists, proceeding with cleaning...


C:\Users\RAMSES\AppData\Local\Temp\ipykernel_20064\843849783.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['reign_start'] = pd.to_datetime(df['date'], errors='coerce')


Data cleaned and saved to 'cleaned_wrestling_data.csv'
